# 2D Heat Equation on a Square with Simple Analytical Comparison

This notebook solves the 2D heat equation on a square domain \([0, 1] \times [0, 1]\) with a simple analytical solution and compares the maximum temperature. The problem is:
\begin{align}
\frac{\partial u}{\partial t} &= \kappa \Delta u, \quad (x, y) \in [0, 1] \times [0, 1], \quad t > 0 \\
u(x, y, 0) &= \sin(\pi x) \sin(\pi y), \\
u &= 0 \quad \text{on } \partial \Omega,
\end{align}
with \( \kappa = 1 \). The analytical solution is:
\[ u(x, y, t) = e^{-2 \pi^2 t} \sin(\pi x) \sin(\pi y) \]
The maximum temperature \( u_{\text{max}}(t) = e^{-2 \pi^2 t} \) (at \( (x, y) = (0.5, 0.5) \)) is compared between numerical and analytical solutions.

In [1]:
import matplotlib as mpl
import pyvista
import ufl
import numpy as np
from petsc4py import PETSc
from mpi4py import MPI
from dolfinx import fem, mesh, io, plot
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc
import matplotlib.pyplot as plt

# Define temporal parameters
t = 0
T = 0.5
num_steps = 50
dt = T / num_steps
kappa = 1.0  # Diffusion coefficient

# Define mesh
nx, ny = 50, 50
domain = mesh.create_rectangle(MPI.COMM_WORLD, [np.array([0, 0]), np.array([1, 1])], [nx, ny], mesh.CellType.quadrilateral)
V = fem.functionspace(domain, ("Lagrange", 1))

The square domain is meshed with quadrilateral elements for better accuracy.

In [2]:
# Create initial condition
def initial_condition(x):
    return np.sin(np.pi * x[0]) * np.sin(np.pi * x[1])

u_n = fem.Function(V)
u_n.name = "u_n"
u_n.interpolate(initial_condition)

# Create boundary condition
fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(
    domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
bc = fem.dirichletbc(PETSc.ScalarType(0), fem.locate_dofs_topological(V, fdim, boundary_facets), V)

## Time-Dependent Output
Store the solution for visualization in Paraview.

In [3]:
xdmf = io.XDMFFile(domain.comm, "diffusion_2d_simple.xdmf", "w")
xdmf.write_mesh(domain)

# Define solution variable
uh = fem.Function(V)
uh.name = "uh"
uh.interpolate(initial_condition)
xdmf.write_function(uh, t)

## Variational Problem
Define the weak form with diffusion coefficient \( \kappa \).

In [4]:
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
f = fem.Constant(domain, PETSc.ScalarType(0))
a = u * v * ufl.dx + kappa * dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = (u_n + dt * f) * v * ufl.dx

## Linear Algebra Structures
Assemble the bilinear and linear forms.

In [5]:
bilinear_form = fem.form(a)
linear_form = fem.form(L)

A = assemble_matrix(bilinear_form, bcs=[bc])
A.assemble()
b = create_vector(linear_form)

## Linear Solver
Use a PETSc KSP solver with LU factorization.

In [6]:
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

## Analytical Solution and Maximum Temperature
Define the analytical solution and compute \( u_{\text{max}}(t) \):
\[ u(x, y, t) = e^{-2 \pi^2 t} \sin(\pi x) \sin(\pi y) \]
\[ u_{\text{max}}(t) = e^{-2 \pi^2 t} \]

In [7]:
def analytical_solution(x, t, kappa=1.0):
    return np.exp(-2 * kappa * np.pi**2 * t) * np.sin(np.pi * x[0]) * np.sin(np.pi * x[1])

def analytical_max_temperature(t, kappa=1.0):
    return np.exp(-2 * kappa * np.pi**2 * t)

## Visualization with PyVista
Create a GIF of the numerical solution.

In [8]:
pyvista.start_xvfb()
grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(V))
plotter = pyvista.Plotter()
plotter.open_gif("u_time_2d_simple.gif", fps=10)
grid.point_data["uh"] = uh.x.array
warped = grid.warp_by_scalar("uh", factor=1)
viridis = mpl.colormaps.get_cmap("viridis").resampled(25)
sargs = dict(title_font_size=25, label_font_size=20, fmt="%.2e", color="black",
             position_x=0.1, position_y=0.8, width=0.8, height=0.1)
renderer = plotter.add_mesh(warped, show_edges=True, lighting=False,
                            cmap=viridis, scalar_bar_args=sargs,
                            clim=[0, max(uh.x.array)])

## Time-Stepping and Comparison
Solve the problem, compute numerical maximum temperature, and compare with analytical.

In [9]:
# Lists to store maximum temperature
times = []
u_max_numerical = []
u_max_analytical = []

for i in range(num_steps):
    t += dt
    
    # Update right-hand side
    with b.localForm() as loc_b:
        loc_b.set(0)
    assemble_vector(b, linear_form)
    apply_lifting(b, [bilinear_form], [[bc]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [bc])
    
    # Solve
    solver.solve(b, uh.x.petsc_vec)
    uh.x.scatter_forward()
    
    # Update previous solution
    u_n.x.array[:] = uh.x.array
    
    # Write to file
    xdmf.write_function(uh, t)
    
    # Update visualization
    new_warped = grid.warp_by_scalar("uh", factor=1)
    warped.points[:, :] = new_warped.points
    warped.point_data["uh"][:] = uh.x.array
    plotter.write_frame()
    
    # Compute maximum temperature
    u_max_num = np.max(uh.x.array)
    u_max_ana = analytical_max_temperature(t, kappa)
    times.append(t)
    u_max_numerical.append(u_max_num)
    u_max_analytical.append(u_max_ana)

plotter.close()
xdmf.close()

# Plot maximum temperature comparison
plt.figure()
plt.plot(times, u_max_numerical, 'o-', label='Numerical')
plt.plot(times, u_max_analytical, 's-', label='Analytical')
plt.xlabel('Time')
plt.ylabel('Maximum Temperature u_max(t)')
plt.legend()
plt.grid(True)
plt.title('Comparison of Maximum Temperature')
plt.show()

# Compute relative error at final time
relative_error = abs(u_max_numerical[-1] - u_max_analytical[-1]) / u_max_analytical[-1]
print(f'Relative error in u_max at t={T}: {relative_error:.2e}')

<img src="./u_time_2d_simple.gif" alt="gif" class="bg-primary mb-1" width="800px">

## Animation with Paraview
![Gif](u_time.gif)Open `diffusion_2d_simple.xdmf` in Paraview, add time annotation, and save the animation as described previously.